In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length","Time on App",
              "Time on Website",'Length of Membership'],
    outputCol="features")

In [12]:
output = assembler.transform(data)

In [13]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [14]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [15]:
final_data = output.select('features','Yearly Amount Spent')

In [16]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                340|
|   mean| 499.12894464135917|
| stddev|   77.8836824571797|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                160|
|   mean|  499.7073621952083|
| stddev|   82.5216114645283|
|    min| 298.76200786180766|
|    max|  725.5848140556806|
+-------+-------------------+



In [19]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [20]:
lrModel = lr.fit(train_data,)

In [21]:
print("Coefficients: {} Intercept: ".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [26.284153158097666,38.420510442779026,0.9976811315187538,61.200774944938594] Intercept: 


In [22]:
test_results = lrModel.evaluate(test_data)

In [23]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 12.173335424254901|
| 12.612546901287374|
|   8.84575481279694|
|  4.401250018168753|
|  4.364938293057662|
| -9.147056929387816|
|  3.558127880449831|
|-0.5353766583667721|
| 19.143932985277843|
| -9.472038385169299|
| -2.042688059002103|
| -16.33212600848134|
|   7.78164611844926|
| -8.846550445040464|
|  9.647058678069527|
| -9.560302350061534|
|  -8.93797199968293|
| -1.214833201197223|
|-3.6812788481157668|
|  5.204136852670217|
+-------------------+
only showing top 20 rows



In [24]:
unlabeled_data = test_data.select('features')

In [25]:
predictions = lrModel.transform(unlabeled_data)

In [26]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 396.4670156483726|
|[30.7377203726281...| 449.1681952949425|
|[30.9716756438877...| 485.7928549440958|
|[31.0472221394875...|388.09614917085264|
|[31.3091926408918...|428.35577954687597|
|[31.5261978982398...| 418.2415831217256|
|[31.5316044825729...|432.95747784891273|
|[31.5761319713222...| 541.7619606476951|
|[31.6005122003032...| 460.0289185058191|
|[31.8093003166791...| 546.2439377480105|
|[31.8124825597242...| 394.8530330427993|
|[31.8164283341993...| 517.4546175121377|
|[31.8209982016720...| 416.8936348947641|
|[31.8279790554652...|  448.849297991982|
|[31.8512531286083...|463.34518798872887|
|[31.8648325480987...| 449.4515828268752|
|[31.8854062999117...|399.04124497215844|
|[31.9120759292006...|388.74954950690494|
|[31.9453957483445...| 660.7012027857677|
|[31.9764800614612...|   325.39030918143|
+--------------------+------------

In [27]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.07429674201034
MSE: 101.49145484608015
